<a href="https://colab.research.google.com/github/sseerrnn/HW-SYN-LAB-FINAL-PROJECT-2022/blob/main/ASR_chief_complain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/biodatlab/whisper-th-demo/blob/main/whisper_th_demo.ipynb)


# **Thonburian Whisper**

Automatic Speech Recognition (ASR) model for Thai

<img src="https://raw.githubusercontent.com/biodatlab/whisper-th-demo/main/assets/Thonburian-Whisper-1.jpg" width="800"/>
---



> By Crews from Looloo Technology and Mahidol University




## **Install Dependencies** ⚙

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!sudo apt install ffmpeg
!pip install torchaudio ipywebrtc notebook
!pip install -q gradio
!pip install pytube
!pip install pythainlp
!pip install marisa_trie
!jupyter nbextension enable --py widgetsnbextension

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7r4gw8_s
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-7r4gw8_s
  Resolved https://github.com/huggingface/transformers to commit 44caf4f6f47120a4aaca561c9cc8041455bef705
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pyp

In [ ]:
import pandas as pd
from pythainlp.tokenize import word_tokenize

In [ ]:
data = pd.read_csv('/content/mockdata.csv')
data

,chief_complain,synonym1,synonym2,synonym3,synonym4,synonym5,synonym6,synonym7
0,กระหายน้ำบ่อย,หิวน้ำบ่อย,หิวน้ำตลอดเวลา,กระหายน้ำมาก,อยากน้ำ,โหยน้ำ,กระหายน้ำมาก,NaN
1,กลืนติด,กลืนแล้วติด,NaN,NaN,NaN,NaN,NaN,NaN
2,กลืนลำบาก,กลืนยาก,กลืนอาหารไม่ได้,กลืนแล้วเจ็บ,NaN,NaN,NaN,NaN
3,การได้ยินลดลง,ไม่ค่อยได้ยิน,หูตึง,เรียกไม่ได้ยิน,หูหนวก,NaN,NaN,NaN
4,ก้อนที่คอ,มีอะไรติดอยู่ที่คอ,ก้อนเนื้อบริเวณลำคอ,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
95,แขนบวม,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,แน่นจมูก,หายใจไม่สะดวก,NaN,NaN,NaN,NaN,NaN,NaN
97,แน่นหน้าอก,เจ็บหน้าอก,NaN,NaN,NaN,NaN,NaN,NaN
98,แสบท้อง,แสบกระเพาะ,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
restructdata = data.copy()
restructdata['synonym']= data['chief_complain'].copy()
restructdata = restructdata[['chief_complain','synonym']]
restructdata

,chief_complain,synonym
0,กระหายน้ำบ่อย,กระหายน้ำบ่อย
1,กลืนติด,กลืนติด
2,กลืนลำบาก,กลืนลำบาก
3,การได้ยินลดลง,การได้ยินลดลง
4,ก้อนที่คอ,ก้อนที่คอ
...,...,...
95,แขนบวม,แขนบวม
96,แน่นจมูก,แน่นจมูก
97,แน่นหน้าอก,แน่นหน้าอก
98,แสบท้อง,แสบท้อง


In [ ]:
for col in data.columns:
  if col != 'chief_complain':
    syn = data[['chief_complain',col]].copy()
    syn = syn.dropna()
    syn.rename(columns={col:'synonym'},inplace= True)
    restructdata= restructdata.append(syn, ignore_index=True)
    
restructdata

,chief_complain,synonym
0,กระหายน้ำบ่อย,กระหายน้ำบ่อย
1,กลืนติด,กลืนติด
2,กลืนลำบาก,กลืนลำบาก
3,การได้ยินลดลง,การได้ยินลดลง
4,ก้อนที่คอ,ก้อนที่คอ
...,...,...
205,คิดวนซ้ำๆ,กังวลอยู่กับเรื่องเดิมวนเวียนไปมา
206,ตาพร่ามัว,ภาพมัว
207,ถ่ายเป็นเลือดสด,เลือดไหลออกมาตอนอุจจาระ
208,กระหายน้ำบ่อย,โหยน้ำ


## **Load and Set-up Thonburian Whisper 🤗**


In [ ]:
import torch
from transformers import pipeline

MODEL_NAME = "biodatlab/whisper-th-medium-combined"
lang = "th"

device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language=lang, task="transcribe")

## **Try it with your own voice** 🎥

### Record your own audio here in the notebook!

In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [ ]:
# Save the recorded audio.
recorder.save("audio.mp3")

ValueError: ignored

### Now let our *Thonburian Whisper* do the work!!

In [ ]:
output = pipe("audio.mp3", ignore_warning = True)
text = output["text"]
text



In [ ]:
text.replace(" ", "")

### Tokenization

In [ ]:
word = restructdata['synonym'].values.tolist()


In [ ]:
from pythainlp.corpus.common import thai_words
import marisa_trie
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
dic = set(CHARS)
dic.update(word)

trie = marisa_trie.Trie(dic)
token = word_tokenize(text, engine="newmm",custom_dict=trie)



In [ ]:
token

### Word Classification

In [ ]:
other = []
for i in CHARS:
  other.append(['other',i])

other = pd.DataFrame(other, columns=['chief_complain','synonym'])
restructdata = restructdata.append(other,ignore_index = True)
restructdata

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(restructdata['synonym'], restructdata['chief_complain'], test_size=0.10, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB())])
text_clf = text_clf.fit(X_train, y_train)

In [ ]:
predicted = text_clf.predict(X_test)
predicted

In [ ]:
predicted = text_clf.predict(token)
predicted